In [1]:
from ultralytics import YOLO
from tqdm import tqdm
import numpy as np
import torch

model = YOLO("runs/detect/yolov11-initial/weights/best.pt") 

In [2]:

metrics = model.val(
    data="coco.yaml",
    split="val",
    imgsz=480,
    conf=0.5,  # Limiar de confiança
    iou=0.6, # Limiar de NMS
)


Ultralytics 8.3.142  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 8192MiB)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 738.7695.8 MB/s, size: 138.6 KB)


val: Scanning E:\COCO-Dataset\val2017\labels.cache... 5000 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [02:59<00:00,  1.74it/s]


                   all       5000      36781      0.623      0.168      0.398      0.296
                person       2693      11004      0.913      0.377      0.651      0.481
               bicycle        149        316      0.653      0.101      0.371      0.227
                   car        535       1932      0.843      0.194      0.513      0.379
            motorcycle        159        371      0.822      0.224      0.527      0.344
              airplane         97        143      0.859      0.427      0.665      0.497
                   bus        189        285      0.779      0.421      0.627      0.561
                 train        157        190      0.828      0.505      0.684      0.574
                 truck        250        415      0.592      0.108      0.353      0.269
                  boat        121        430      0.614     0.0814      0.341      0.175
         traffic light        191        637      0.708      0.107      0.402      0.226
          fire hydran

In [3]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001B613203620>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0

In [2]:
def calculate_entropy(probabilities):
    """Calcula a entropia de Shannon para um vetor de probabilidades."""
    epsilon = 1e-10  # Evitar log(0)
    return -np.sum(probabilities * np.log(probabilities + epsilon), axis=-1)

In [ ]:
from cocoValDataset import CocoValDataset
from torch.utils.data import DataLoader

def validate_model(model, device, data_path, batch_size=8):
    dataset = CocoValDataset(data_path)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    all_confs = []
    all_entropies = []
    
    # Configurações do progresso
    pbar_format = "Validando: {percentage:.0f}%|{bar}| {n_fmt}/{total_fmt} [Tempo: {elapsed}<{remaining}]"
    
    with torch.no_grad():
        with tqdm(total=len(loader), 
                 desc='Validação', 
                 bar_format=pbar_format,
                 dynamic_ncols=True,  # Ajusta automaticamente ao tamanho do notebook
                 mininterval=0.5,     # Atualiza a cada 0.5 segundos
                 leave=True) as pbar:
            
            for batch, img_names in loader:
                batch = batch.to(device)
                print(img_names)
                
                # Inferência do modelo
                results = model(batch, save_conf=True)
                
                # Processamento dos resultados
                for result in results:
                    if hasattr(result, 'boxes') and result.boxes is not None:
                        confs = result.boxes.conf.cpu().numpy()
                        entropies = [calculate_entropy(c) for c in confs]
                        
                        all_confs.extend(confs.tolist())
                        all_entropies.extend(entropies)
                
            # Atualização dinâmica da barra
            pbar.update(1)
            pbar.refresh()  # Força atualização imediata
    
    # Cálculo final das métricas
    avg_conf = np.mean(all_confs) if all_confs else 0
    avg_entropy = np.mean(all_entropies) if all_entropies else 0
    
    print(f'\n✅ Validação Completa:')
    print(f'  - Confiança Média: {avg_conf:.4f}')
    print(f'  - Entropia Média: {avg_entropy:.4f}')
    
    return avg_conf, avg_entropy


model.to('cuda')
    
data_path = 'F:/COCO-Dataset/train2017/'
avg_conf, avg_entropy = validate_model(model, 'cuda', data_path)

Validando: 0%|          | 0/11826 [Tempo: 00:00<?]

('000000000009.jpg', '000000000025.jpg', '000000000030.jpg', '000000000034.jpg', '000000000042.jpg', '000000000049.jpg', '000000000064.jpg', '000000000072.jpg')

0: 480x480 2 bananas, 1 sandwich, 2 broccolis, 1 donut, 50.6ms
1: 480x480 1 giraffe, 50.6ms
2: 480x480 1 vase, 50.6ms
3: 480x480 1 zebra, 50.6ms
4: 480x480 (no detections), 50.6ms
5: 480x480 3 persons, 1 horse, 50.6ms
6: 480x480 1 car, 1 clock, 50.6ms
7: 480x480 3 giraffes, 50.6ms
Speed: 0.0ms preprocess, 50.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 480)
('000000000077.jpg', '000000000078.jpg', '000000000081.jpg', '000000000086.jpg', '000000000089.jpg', '000000000094.jpg', '000000000109.jpg', '000000000110.jpg')

0: 480x480 2 persons, 53.1ms
1: 480x480 1 clock, 53.1ms
2: 480x480 1 airplane, 53.1ms
3: 480x480 2 persons, 1 bicycle, 1 motorcycle, 53.1ms
4: 480x480 2 ovens, 53.1ms
5: 480x480 2 cars, 1 truck, 1 traffic light, 53.1ms
6: 480x480 3 persons, 53.1ms
7: 480x480 6 persons, 1 pizza, 1 chair, 2 dining 

Validando: 0%|          | 0/11826 [Tempo: 00:24<?]


KeyboardInterrupt: 

In [ ]:
print(avg_conf, avg_entropy)